In [1]:
import numpy as np
import pandas as pd
import json
import sys
import mapping as mp
import logging
sys.path.append("./")

In [8]:
df = pd.read_parquet("C:/Users/akmar/PycharmProjects/lolpowerrank/riot_esports_ranking/df_stacked.parquet")

In [58]:
teams = df['team'].unique().tolist()

In [15]:
# Extract unique team-region pairs and set 'team' as the index
team_region_df = df[['team', 'Region']].drop_duplicates().set_index('team')

# Create the elo_map dictionary
elo_map = team_region_df['Region'].to_csv("team_to_region.csv")

In [11]:
a=set(elo_map.values())

In [32]:
df = pd.read_csv('C:/Users/akmar/PycharmProjects/lolpowerrank/riot_esports_ranking/DataPrep/team_to_region.csv')
dict(df[['team','elo']].set_index('team')['elo'])

{'mirage-alliance': 1200,
 'return-of-the-middlesticks': 1200,
 'gentle-hearts-gaming': 1200,
 'komodo': 1200,
 'team-ambition': 1200,
 'team-coachify': 1200,
 'teamless-revenge': 1200,
 'kelyxs-grandpa-gamers': 1200,
 'winthrop-university': 1200,
 'cb-gaming': 1200,
 'team-e-turner': 1200,
 'aporia': 1200,
 'cold-hearted': 1200,
 'lit-esports': 1200,
 'ccg-esports': 1200,
 'apex-mission-impossible': 1200,
 'team-phantasma': 800,
 'anorthosis-famagusta-esports': 800,
 'gamespace-mediterranean-college-esports': 800,
 'panathinaikos-esports': 800,
 'wild-panther-esports': 800,
 'zerolag-esports': 800,
 'we-love-gaming': 800,
 'pentagon-rejects': 800,
 'heet': 800,
 'a-one-man-army': 800,
 'myth-esports': 800,
 'krc-genk-esports': 800,
 'northern-lions-esports': 800,
 'zennit-gaming': 800,
 'mcon-lg-ultragear': 800,
 'team-7am': 800,
 'giants': 800,
 'fnatictq': 800,
 'team-heretics-sl': 800,
 'barca-esports': 800,
 'finetwork-koi': 800,
 'ucam-esports-club': 800,
 'rebels-gaming': 800,
 

In [42]:
df.pivot(index='platformgameid', columns='player_id', values='assists')


player_id,1,2,3,4,5,6,7,8,9,10
platformgameid,,,,,,,,,,
ESPORTSTMNT01:1110148,7.0,19.0,12.0,8.0,14.0,3.0,4.0,1.0,0.0,6.0
ESPORTSTMNT01:1110162,4.0,5.0,12.0,4.0,9.0,0.0,6.0,3.0,4.0,6.0
ESPORTSTMNT01:1110177,3.0,9.0,5.0,6.0,13.0,1.0,3.0,5.0,0.0,1.0
ESPORTSTMNT01:1110187,7.0,21.0,9.0,13.0,17.0,3.0,8.0,8.0,8.0,11.0
ESPORTSTMNT01:1110198,7.0,14.0,6.0,6.0,13.0,2.0,8.0,5.0,4.0,9.0
...,...,...,...,...,...,...,...,...,...,...
LPL_B:330027,5.0,7.0,2.0,6.0,8.0,1.0,5.0,4.0,3.0,9.0
LPL_B:330032,3.0,6.0,2.0,2.0,7.0,6.0,9.0,4.0,1.0,9.0
LPL_B:330033,11.0,15.0,13.0,11.0,23.0,5.0,7.0,10.0,9.0,12.0


In [2]:

import Modelling.Simple.elo as elo

# Create a logger
logger = logging.getLogger('mylogger')
logger.setLevel(logging.DEBUG)

# Create a file handler
fh = logging.FileHandler('mylog.log')
fh.setLevel(logging.DEBUG)

# Create a console handler
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)  

In [4]:
### Run this to get data to upload to S3 bucket to continue analysis and modelling

# Load all the data
filepath = f"C:/Users/akmar/PycharmProjects/lolpowerrank/riot_esports_ranking/esports-data/"
map_data = mp.game_mapping(filepath=filepath)
map_data = mp.player_mapping(map_data,filepath)
map_data = mp.team_mapping(map_data,filepath)
league_data = mp.league_mapping(filepath)
tournament_data = mp.tournament_mapping(filepath)
# Merges
tournament_data =  tournament_data.merge(map_data,how="left",left_on='game_id',right_on='esportsGameId')
tournament_data = tournament_data.merge(league_data,how="left",on='tournament_id')
#tournament_data.to_parquet("combined_data_raw.parquet")
# Data Validation
t_d_unique = set(tournament_data['game_id'])
m_d_unique = set(map_data['esportsGameId'])
t_d_diff = t_d_unique.difference(m_d_unique)
m_d_diff = m_d_unique.difference(t_d_unique)
logger.info(f"Game_id in Tournament Data not in Mapping Data: {t_d_diff}")
logger.info(f"esportsGameId in Mapping Data not in Tournament Data: {m_d_diff}")
t_d_unique_2 = set(tournament_data['tournament_id'])
l_d_unique_2 = set(league_data['tournament_id'])

In [7]:
map_data[map_data['esportsGameId'].isin(list(m_d_diff))].head()

,esportsGameId,platformGameId,teammapping_200,teammapping_100,participantmapping_3,participantmapping_5,participantmapping_10,participantmapping_2,participantmapping_1,participantmapping_9,...,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,team_100,team_200
39,102147203803367589,ESPORTSTMNT03:1114809,98767991935149427,99566405944694628,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pixel-esports-club,rainbow7
51,102181142509022803,ESPORTSTMNT01:1155853,99566404581868574,98767991853197861,98767991747728851,99566404535234175,NaN,99566404797900723,98767975939657196,99566404541006806,...,Faker,Teddy,Effort,Kiin,Dread,Ucal,Aiming,NaN,t1,kwangdong-freecs
62,102147201397267904,ESPORTSTMNT02:1114771,99294153826603341,98926509885559666,98767991798919851,98767991789638893,99101098208825497,98767991759219370,98767991746528652,98767975927317955,...,Jensen,Doublelift,CoreJJ,Huni,Lira,Damonte,Cody Sun,Vulcan,team-liquid,clutch-gaming
68,102181142512430702,ESPORTSTMNT02:1113579,100725845018863243,98767991853197861,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t1,dwg-kia
73,101978171972931515,ESPORTSTMNT01:1121486,99566404848691211,98767991853197861,98767991747728851,99566404529717522,99566404837520336,99566404797900723,98767975939657196,99566404823220763,...,Faker,Teddy,Mata,TheShy,Ning,Rookie,JackeyLove,Baolan,t1,invictus-gaming
